In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Set up file client
account_name = "phdidevphi9d194c64"
file_system_name = "source-data"
file_path = "synthetic_patient_mci_seed_data.parquet"
mci_incoming_file_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{file_path}"
mci_delta_table_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/delta/mci"


def insert(mci_incoming_file_path,mci_delta_table_path):
    df = spark.read.parquet(mci_incoming_file_path)
    df.write.mode("append").format("parquet").save(mci_delta_table_path)
    spark.read.parquet(mci_delta_table_path).show(5)

insert(mci_incoming_file_path,mci_delta_table_path)